In [1]:
#保存するディレクトリ
dir = 'takamatsu/horsepage/2006'

In [2]:
#raceのURL
RACE_URL = "https://db.netkeiba.com/race/200607010811/"

予測したいレースに関連する馬をスクレイピングするコード

In [3]:
#インポート文
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import os
import time
import requests

import re

In [4]:
#ドライバーの設定
driver = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(driver,10)

C:\Users\kawam\AppData\Local\Temp\ipykernel_3840\2084027807.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [5]:
#ドライバー起動
driver.get(RACE_URL)
time.sleep(1)
wait.until(EC.presence_of_all_elements_located)

<function selenium.webdriver.support.expected_conditions.presence_of_all_elements_located.<locals>._predicate(driver)>

In [6]:
horse_href_list=[]
all_rows = driver.find_element(By.CLASS_NAME,'race_table_01').find_elements(By.TAG_NAME,"tr")
for row in range(1, len(all_rows)):
    horse_href=all_rows[row].find_elements(By.TAG_NAME,"td")[3].find_element(By.TAG_NAME,"a").get_attribute("href")
    horse_href_list.append(horse_href)

In [7]:
HTML_DIR = "html/"+dir
if not os.path.isdir(HTML_DIR):
    os.makedirs(HTML_DIR)
        
for url in horse_href_list:
    list = url.split("/")
    horse_id = list[-2]
    save_file_path = HTML_DIR+"/"+horse_id+'.html'
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    html = response.text
    time.sleep(5)
    with open(save_file_path, 'w') as file:
        file.write(html)

In [8]:
horse_data_info_columns=[
    'horse_id',
    'bday',
    'tame_id',
    'owner_id',
    'producer_id',
    'production area',
    'auction price',
    'winnings',
    'lifetime record',
    'wined race title',
    'inbreeding-1',
    'inbreeding-2',
    'father',
    'faths father',
    'faths mother',
    'mother',
    'moths father',
    'moths mother'
]

In [9]:
horse_race_columns=["date",
                    "whereracecourse",
                    "weather",
                    "race_round",
                    "race_title",
                    "race_id",
                    "total_horse_number",
                    "frame_number",
                    "horse_number",
                    "odds",
                    "popular",
                    "rank",
                    "rider_id",
                    "horse_wight",
                    "distance",
                    "groud_status",
                    'goal_time',
                    'goal_time_dif',
                    'half_way_rank',
                    'pace',
                    'last_time',
                    'horse_weight',
                    'runner-up-horse-id',
                    'prize'
                    ]

In [10]:
def get_horse_data_by_html(horse_id, html):
    horse_list = [horse_id]
    horse_race_list = []
    soup = BeautifulSoup(html, 'html.parser')
    horse_tables = soup.find("table", class_="db_prof_table")
    horse_table = horse_tables.findAll("td")
    #for i in range(10):
    horse_list.append(horse_table[0].get_text())
    horse_list.append(horse_table[1].find('a').get('href').split("/")[-2])
    horse_list.append(horse_table[2].find('a').get('href').split("/")[-2])
    horse_list.append(horse_table[3].find('a').get('href').split("/")[-2])
    horse_list.append(horse_table[4].get_text())
    horse_list.append(horse_table[5].get_text())
    horse_list.append(horse_table[6].get_text())
    horse_list.append(horse_table[7].get_text())
    horse_list.append(horse_table[8].find('a').get('href').split("/")[-2])
    
    # horse_table[8]からすべてのリンクのURLの一部を取得する
    url_parts = [link.get('href').split("/")[-2] for link in horse_table[9].find_all('a')]

    # url_partsの要素数が2未満の場合、残りの要素をNaNで埋める
    while len(url_parts) < 2:
        url_parts.append(np.nan)

    # horse_listにurl_partsを追加する
    horse_list.extend(url_parts)
    
    #horse_list.append(horse_table[9].get_text())
    blood_tables = soup.find("table", class_="blood_table")
    blood_table = blood_tables.findAll("td")
    #for i in range(6):
    horse_list.append(blood_table[0].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[1].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[2].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[3].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[4].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[5].find('a').get('href').split("/")[-2])    
    
    
    
    
    
#horse_race
    horse_tables = soup.find("table", class_="db_h_race_results")
    horse_trs = horse_tables.findAll("tr")
    for index, horse_tr in enumerate(horse_trs):
        if index == 0:
            continue
        horse_tds = horse_tr.findAll("td")
        if horse_tds[26].find('a') is not None:
            horse_race_list.append([horse_tds[0].get_text(),
                                    horse_tds[1].get_text(),
                                    horse_tds[2].get_text(),
                                    horse_tds[3].get_text(),
                                    horse_tds[4].get_text(),
                                    horse_tds[4].find('a').get('href').split("/")[-2],
                                    horse_tds[6].get_text(),
                                    horse_tds[7].get_text(),
                                    horse_tds[8].get_text(),
                                    horse_tds[9].get_text(),
                                    horse_tds[10].get_text(),
                                    horse_tds[11].get_text(),
                                    horse_tds[12].find('a').get('href').split("/")[-2],
                                    horse_tds[13].get_text(),
                                    horse_tds[14].get_text(),
                                    horse_tds[15].get_text(),
                                    horse_tds[17].get_text(),
                                    horse_tds[18].get_text(),
                                    horse_tds[20].get_text(),
                                    horse_tds[21].get_text(),
                                    horse_tds[22].get_text(),
                                    horse_tds[23].get_text(),
                                    horse_tds[26].find('a').get('href').split("/")[-2],
                                    horse_tds[27].get_text(),
                                   ])
        else:
            horse_race_list.append([horse_tds[0].get_text(),
                                    horse_tds[1].get_text(),
                                    horse_tds[2].get_text(),
                                    horse_tds[3].get_text(),
                                    horse_tds[4].get_text(),
                                    horse_tds[4].find('a').get('href').split("/")[-2],
                                    horse_tds[6].get_text(),
                                    horse_tds[7].get_text(),
                                    horse_tds[8].get_text(),
                                    horse_tds[9].get_text(),
                                    horse_tds[10].get_text(),
                                    horse_tds[11].get_text(),
                                    horse_tds[12].find('a').get('href').split("/")[-2],
                                    horse_tds[13].get_text(),
                                    horse_tds[14].get_text(),
                                    horse_tds[15].get_text(),
                                    horse_tds[17].get_text(),
                                    horse_tds[18].get_text(),
                                    horse_tds[20].get_text(),
                                    horse_tds[21].get_text(),
                                    horse_tds[22].get_text(),
                                    horse_tds[23].get_text(),
                                    None,
                                    horse_tds[27].get_text(),
                                   ])
                                        
    horse_race_tmp_df = pd.DataFrame(horse_race_list, columns=horse_race_columns)
    horse_race_tmp_df.loc[:, 'horse_id'] = horse_id
    
    return horse_list , horse_race_tmp_df

In [11]:
CSV_DIR = "csv/"+dir
if not os.path.isdir(CSV_DIR):
    os.makedirs(CSV_DIR)
horse_info_csv = CSV_DIR+"/horse-info.csv"
horse_race_csv = CSV_DIR+"/horse-race.csv"
horse_info_df = pd.DataFrame(columns=horse_data_info_columns)
horse_race_df = pd.DataFrame()
if os.path.isdir(HTML_DIR):
    file_list = os.listdir(HTML_DIR)
    for file_name in file_list:
        with open(HTML_DIR+"/"+file_name, "r") as f:
            html = f.read()
            list = file_name.split(".")
            horse_id = list[-2]
            horse_list , horse_race_tmp_df = get_horse_data_by_html(horse_id, html) 
            horse_se = pd.Series(horse_list, index=horse_info_df.columns)
            horse_info_df = pd.concat([horse_info_df, horse_se.to_frame().T], ignore_index=True)
            horse_race_df = pd.concat([horse_race_df, horse_race_tmp_df], axis=0, ignore_index=True)       

### cleaning

##### horse_race

In [18]:
horse_info_df.head(1)

,horse_id,bday,tame_id,owner_id,producer_id,production area,auction price,winnings,lifetime record,wined race title,inbreeding-1,inbreeding-2,father,faths father,faths mother,mother,moths father,moths mother
0,1998107227,1998年4月3日,05489,083030,700317,新冠,"1,302万円 (1999年 北海道7月セレクト市場)\n","\n\n\n\n2億9,890万円 (中央) /2,040万円 (地方)\n",52戦7勝 [7-6-5-34],200606020311,1990103624,1989102140,1988109108,000a001bda,000a009781,1981106025,000a000258,000a00075e
1,1999100510,1999年5月2日,00221,188800,900085,静内町,\n-\n,"\n\n\n2億2,645万円 (中央)\n\n",68戦7勝 [7-6-5-50],200609010211,2003104986,2008100457,1989108341,1982101222,1984104366,1993106753,1985104409,1988101680
2,1999102397,1999年5月30日,01064,914800,000348,門別町,\n-\n,"\n\n\n1億3,392万円 (中央)\n\n",44戦6勝 [6-4-2-32],200506040711,1994102639,1998105635,000a000354,000a0016eb,000a00a4ad,1986101925,000a000166,1978105201
3,1999106730,1999年6月5日,00425,415800,393126,千歳市,\n-\n,"\n\n\n\n2億9,956万円 (中央) /2億6,906万円 (地方)\n",63戦14勝 [14-8-10-31],200644100410,1998101530,1997103416,000a00033a,000a0012bf,000a008c1e,1989107932,000a000258,000a006149
4,1999106907,1999年3月13日,01017,172002,343126,白老町,840万円 (1999年 セレクトセール)\n,"\n\n\n\n4億2,690万円 (中央) /8,140万円 (地方)\n",44戦14勝 [14-10-2-18],200608050711,1998107262,2001103268,1992102988,000a000082,000a00617d,1992109451,000a00033a,1980101955
5,1999110175,1999年2月17日,00427,951005,200879,米,\n-\n,"\n\n\n\n2億9,725万円 (中央) /403万円 (地方)\n",52戦8勝 [8-4-3-37],200408020411,000a006643,,000a002161,000a001607,000a00ada1,000a00a808,000a0012cb,000a00a807
6,2000101550,2000年1月16日,01002,334009,370126,早来町,\n-\n,"\n\n\n3億8,457万円 (中央)\n\n",38戦9勝 [9-8-5-16],200607010811,1995107881,2004102874,000a00033a,000a0012bf,000a008c1e,1990104543,000a000d30,1980101983
7,2000102386,2000年4月2日,00244,078800,910082,静内町,\n-\n,\n\n\n5億449万円 (中央)\n\n,38戦7勝 [7-8-5-18],200307030611,2005100664,2007100618,1989108341,1982101222,1984104366,1996105163,1988106095,1977106216
8,2000102393,2000年6月3日,00390,078800,910082,静内町,\n-\n,"\n\n\n\n2億3,405万円 (中央) /810万円 (地方)\n",38戦11勝 [11-3-4-20],200504030711,2005100670,1997102415,000a00025e,000a0010a8,000a0093ef,1992102137,000a000249,1980100896
9,2000106626,2000年4月29日,00365,609009,930321,門別町,378万円 (2001年 北海道10月市場)\n,"\n\n\n2億8,878万円 (中央)\n\n",42戦6勝 [6-5-2-29],200308040711,1999103190,1995103274,1988107894,000a00175b,000a00006a,1988103143,000a0001eb,000a0005fa


##### date

In [19]:
horse_race_df.head(1)

,date,whereracecourse,weather,race_round,race_title,race_id,total_horse_number,frame_number,horse_number,odds,...,groud_status,goal_time,goal_time_dif,half_way_rank,pace,last_time,horse_weight,runner-up-horse-id,prize,horse_id
0,2007/12/26,浦和,晴,11,テレビ埼玉杯(G),200742122611,11,1,1,63.1,...,重,1:28.2,1.3,5-5-7-6,35.5-38.7,39.5,501(0),2000104545,,1998107227
1,2007/11/30,大井,曇,11,アフター5スター賞競(G),200744113011,16,6,12,55.6,...,良,1:13.9,2.5,4-4,34.5-36.9,38.9,501(+17),2001102299,,1998107227
2,2007/10/27,4京都8,晴,11,毎日放送賞スワンS(G2),200708040811,18,7,15,156.1,...,稍,1:21.9,1.2,11-15,33.7-35.5,35.0,484(-22),2003101859,,1998107227
3,2007/10/03,大井,曇,11,東京盃競走(G2),200744100311,16,3,5,110.8,...,重,1:12.6,1.3,6-9,34.3-37.0,37.9,506(+22),1999106730,,1998107227
4,2007/06/12,旭川,晴,10,北海道スプリントカッ(G3),200734061210,14,6,10,28.6,...,良,1:01.2,1.2,9-10,,37.2,484(-10),2002100683,,1998107227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830,2004/12/12,5中山4,曇,3,2歳未勝利,200406050403,18,7,13,9.4,...,良,2:03.3,0.2,5-6-6-5,36.2-37.1,36.6,514(+2),2002110023,200.0,2002110074
831,2004/11/20,5東京5,晴,3,2歳未勝利,200405050503,16,7,14,9.8,...,稍,2:03.0,0.1,1-1-1,36.2-37.3,37.4,512(-2),2002102046,75.0,2002110074
832,2004/11/06,5東京1,曇,3,2歳未勝利,200405050103,13,3,3,20.9,...,良,2:03.3,0.2,4-4-4,36.0-36.0,35.7,514(+6),2002103426,130.0,2002110074
833,2004/10/23,4東京5,晴,2,2歳未勝利,200405040502,15,7,13,6.5,...,良,1:24.8,1.8,12-14,35.2-35.4,35.8,508(-6),2002110026,,2002110074


In [20]:
horse_race_df['date'] = pd.to_datetime(horse_race_df['date'], format='%Y/%m/%d')

##### race_title

In [21]:
# 'race_title' 列から () の中身を抽出する
horse_race_df['race_type'] = horse_race_df['race_title'].str.extract('\((.*?)\)')

# 'race_type' 列の欠損値を空文字列に置き換える
horse_race_df['race_type'].fillna('', inplace=True)

##### odds

In [23]:
#数字以外の文字を削除
horse_race_df['odds'] = horse_race_df['odds'].apply(lambda x: re.sub('[^0-9\.]', '', x))
#空白を削除
horse_race_df['odds'] = horse_race_df['odds'].replace('', np.nan)
#欠損値の行を削除
horse_race_df = horse_race_df.dropna(subset=['odds'])

In [24]:
horse_race_df['odds'] = horse_race_df['odds'].astype(float)

C:\Users\kawam\AppData\Local\Temp\ipykernel_3840\2488911290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horse_race_df['odds'] = horse_race_df['odds'].astype(float)


##### rank

In [25]:
#数字以外の文字を含む列を削除
horse_race_df = horse_race_df[pd.to_numeric(horse_race_df['rank'], errors='coerce').notnull()]
#数値に変換
horse_race_df['rank'] = horse_race_df['rank'].astype(int)

##### distance

In [26]:
#文字と数値に分離させる
horse_race_df[['surface', 'distance']] = horse_race_df['distance'].str.extract('(\D+)(\d+)')
# 'distance' 列を整数値に変換する
horse_race_df['distance'] = horse_race_df['distance'].astype(int)

##### goal_time

In [27]:
#数値に変換
def to_seconds(time_str):
    minutes, seconds = map(float, time_str.split(':'))
    return minutes * 60 + seconds

horse_race_df['goal_time'] = horse_race_df['goal_time'].apply(to_seconds)

##### goal_time_dif

In [29]:
horse_race_df['goal_time_dif'] = horse_race_df['goal_time_dif'].astype(float)

In [30]:
horse_race_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 812 entries, 0 to 834
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                812 non-null    datetime64[ns]
 1   whereracecourse     812 non-null    object        
 2   weather             812 non-null    object        
 3   race_round          812 non-null    object        
 4   race_title          812 non-null    object        
 5   race_id             812 non-null    object        
 6   total_horse_number  812 non-null    object        
 7   frame_number        812 non-null    object        
 8   horse_number        812 non-null    object        
 9   odds                812 non-null    float64       
 10  popular             812 non-null    object        
 11  rank                812 non-null    int32         
 12  rider_id            812 non-null    object        
 13  horse_wight         812 non-null    object        

In [33]:
horse_info_df.to_csv(horse_info_csv, header=True, index=False)
horse_race_df.to_csv(horse_race_csv, header=True, index=False)

##### 以下メモ

In [31]:
# horse_info_df['horse_id'] = horse_info_df['horse_id']
# horse_info_df['tame_id'] = horse_info_df['tame_id']
# horse_info_df['owner_id'] = horse_info_df['owner_id']

#以下文字列の可能性あり
# horse_info_df['inbreeding-1'] = horse_info_df['inbreeding-1']
# horse_info_df['inbreeding-2'] = horse_info_df['inbreeding-2']
# horse_info_df['father'] = horse_info_df['father']
# horse_info_df['mother'] = horse_info_df['mother']

In [32]:
# # データを読み込む
# data = pd.read_csv('data.csv')

# # 日付でソートする
# data = data.sort_values('date')

# # 同じ ID を持つデータをグループ化する
# grouped_data = data.groupby('ID')

# # 各グループの最初のデータを選択する
# latest_data = grouped_data.last()

In [34]:
# horse_list = [horse_id]
# horse_race_list = []
# soup = BeautifulSoup(html, 'html.parser')
# horse_tables = soup.find("table", class_="db_h_race_results")
# horse_trs = horse_tables.findAll("tr")
# for index, horse_tr in enumerate(horse_trs):
#     if index == 0:
#         continue
#     horse_tds = horse_tr.findAll("td")
#     horse_race_list.append([horse_tds[0].get_text(),
#                             horse_tds[1].get_text(),
#                             horse_tds[2].get_text(),
#                             horse_tds[3].get_text(),
#                             horse_tds[4].find('a').get('href').split("/")[-2],
#                             horse_tds[6].get_text(),
#                             horse_tds[7].get_text(),
#                             horse_tds[8].get_text(),
#                             horse_tds[9].get_text(),
#                             horse_tds[10].get_text(),
#                             horse_tds[11].get_text(),
#                             horse_tds[12].find('a').get('href').split("/")[-2],
#                             horse_tds[13].get_text(),
#                             horse_tds[14].get_text(),
#                             horse_tds[15].get_text(),
#                             horse_tds[17].get_text(),
#                             horse_tds[18].get_text(),
#                             horse_tds[20].get_text(),
#                             horse_tds[21].get_text(),
#                             horse_tds[22].get_text(),
#                             horse_tds[23].get_text(),
#                             horse_tds[26].find('a').get('href').split("/")[-2],
#                             horse_tds[27].get_text(),
#                            ])

# df = pd.DataFrame(horse_race_list, columns=horse_race_columns)
# df.loc[:, 'horse_id'] = horse_id